In [ ]:
import pandas as pd
df = pd.read_csv('/content/spam.csv',encoding='ISO-8859-1')
df.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.shape

(5572, 5)

In [ ]:
df.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [ ]:
df.isnull().sum()

,0
v1,0
v2,0
Unnamed: 2,5522
Unnamed: 3,5560
Unnamed: 4,5566


In [ ]:
df['v1'].value_counts()

,count
v1,
ham,4825
spam,747


In [ ]:
# Keep only the first two useful columns
df = df[['v1', 'v2']]

# Rename columns for clarity
df.columns = ['label', 'message']

# Check again
df.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# Convert message to lowercase
df['message'] = df['message'].str.lower()

# Check few rows
df['message'].head()


,message
0,"go until jurong point, crazy.. available only ..."
1,ok lar... joking wif u oni...
2,free entry in 2 a wkly comp to win fa cup fina...
3,u dun say so early hor... u c already then say...
4,"nah i don't think he goes to usf, he lives aro..."


In [ ]:
import string

#  Remove punctuation
df['message'] = df['message'].apply(lambda x: ''.join(char for char in x if char not in string.punctuation))

# Check a few cleaned rows
df['message'].head()


,message
0,go until jurong point crazy available only in ...
1,ok lar joking wif u oni
2,free entry in 2 a wkly comp to win fa cup fina...
3,u dun say so early hor u c already then say
4,nah i dont think he goes to usf he lives aroun...


In [ ]:
# : Tokenize each message (split into words)
df['message'] = df['message'].apply(lambda x: x.split())

#Check how it looks now
df['message'].head()


,message
0,"[go, until, jurong, point, crazy, available, o..."
1,"[ok, lar, joking, wif, u, oni]"
2,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,"[u, dun, say, so, early, hor, u, c, already, t..."
4,"[nah, i, dont, think, he, goes, to, usf, he, l..."


In [ ]:
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Set of English stopwords
stop_words = set(stopwords.words('english'))



In [ ]:
# Remove stopwords
df['message'] = df['message'].apply(lambda x: [word for word in x if word not in stop_words])



In [ ]:
df['message'].head()

,message
0,"[go, jurong, point, crazy, available, bugis, n..."
1,"[ok, lar, joking, wif, u, oni]"
2,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,"[u, dun, say, early, hor, u, c, already, say]"
4,"[nah, dont, think, goes, usf, lives, around, t..."


In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()


In [ ]:
df['message'] = df['message'].apply(lambda x: [stemmer.stem(word) for word in x])


In [ ]:
df['cleaned_message'] = df['message'].apply(lambda x: ' '.join(x))


In [ ]:
df[['cleaned_message']].head()


,cleaned_message
0,go jurong point crazi avail bugi n great world...
1,ok lar joke wif u oni
2,free entri 2 wkli comp win fa cup final tkt 21...
3,u dun say earli hor u c alreadi say
4,nah dont think goe usf live around though


In [ ]:
# for feature engineering this time we will used word embediing
!wget --no-check-certificate http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip


--2025-06-20 19:37:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-06-20 19:37:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-06-20 19:37:05--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zi

In [ ]:
import numpy as np
embedding_index = {}

with open("glove.6B.100d.txt", encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        embedding_index[word] = vector


In [ ]:
import numpy as np

def get_average_embedding(message):
    words = message.split()
    valid_vectors = [embedding_index[word] for word in words if word in embedding_index]
    if valid_vectors:
        return np.mean(valid_vectors, axis=0)
    else:
        return np.zeros(100)

# Apply to all messages
X = np.array([get_average_embedding(msg) for msg in df['cleaned_message']])


In [ ]:
y = df['label'].map({'ham': 0, 'spam': 1})


In [ ]:
#training and testing split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Train the SVM model
svm_model = SVC(kernel='linear')  # use linear kernel for text
svm_model.fit(X_train, y_train)

# Predict
y_pred_svm = svm_model.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


[[926  40]
 [ 41 108]]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       966
           1       0.73      0.72      0.73       149

    accuracy                           0.93      1115
   macro avg       0.84      0.84      0.84      1115
weighted avg       0.93      0.93      0.93      1115



In [ ]:
# creating a function which will classify it
def preprocess_input(message):
    # Step-by-step preprocessing
    import string
    from nltk.stem.porter import PorterStemmer
    from nltk.corpus import stopwords

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    # Lowercase
    message = message.lower()
    # Remove punctuation
    message = ''.join([char for char in message if char not in string.punctuation])
    # Tokenize
    words = message.split()
    # Remove stopwords and apply stemming
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

def get_glove_vector(text):
    words = text.split()
    valid_vectors = [embedding_index[word] for word in words if word in embedding_index]
    if valid_vectors:
        return np.mean(valid_vectors, axis=0).reshape(1, -1)
    else:
        return np.zeros((1, 100))

def classify_message(message):
    cleaned = preprocess_input(message)
    vector = get_glove_vector(cleaned)
    prediction = svm_model.predict(vector)[0]
    return "Spam " if prediction == 1 else "Safe !! Not Spam "


In [ ]:
msg1 = "Congratulations! You've won a free ticket."
msg2 = "Hey are we still meeting at 5?.click for better response give 50rs"

print("Message 1:", classify_message(msg1))
print("Message 2:", classify_message(msg2))


Message 1: Spam 
Message 2: Safe !! Not Spam 


In [ ]:
import pickle

# Save model
with open("model.pkl", "wb") as f:
    pickle.dump(svm_model, f)

# Save glove embeddings
with open("glove_vectors.pkl", "wb") as f:
    pickle.dump(embedding_index, f)
